<a href="https://colab.research.google.com/github/tendai-codes/DeepLearning/blob/main/Multi_class_Classification_workflow_Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Make Classsification data

In [ ]:
import sklearn

In [ ]:
from sklearn.datasets import fetch_rcv1
rcv1 = fetch_rcv1()

rcv1

In [ ]:
import pandas as pd
data = pd.DataFrame(rcv1.data)
data.head()

### Import Dependencies


In [ ]:
# Import Libraries
import torch
import matplotlib.pyplot as plt
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split

# Set Hyperparameters for data
NUM_CLASSES = 5
NUM_FEATURES = 30
LEARNING_RATE = 0.01
NUM_EPOCHS = 500
RANDOM_SEED = 42


### Turn data into tensors

In [ ]:
# Create multi-class data
X , y = "dataset"(n_samples=1000,
                            n_features=NUM_FEATURES,
                            centres=NUM_CLASSES,
                            cluster_std=1.5,
                            random_state=RANDOM_SEED)

# Convert data to tensors
X = torch.from_numpy(X).type(torch.float)
y = torch.from_numpy(y).type(torch.LongTensor)

X[:5], y[:5]

##Check input and output shapes

In [ ]:
X.shape , y.shape # This will help you figure out input and output features for buildng model

## Split Training an test data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)
#0.2 = 20% of data will be test, 80% will be train

In [ ]:
len(X_train), len(X_test), len(y_train), len(y_test)

### Create Device agnostic code

In [ ]:
# Setup device Agnostic Code
from torch import nn

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using Device: {device}")

Using Device: cpu


### Build Model

In [ ]:
# Create linear regression model class > THIS METHOD FOR LESS COMPLEX LAYERS
class ClassificationModel(nn.Module):
  def __init__(self, input_features, output_features, hidden_units=8):
    """ Initializes multi-calss classification model.

    Args:
    input_features (int): Number of input features to the model.
    output_features (int): Number of output features of the model.
    hidden_units (int): Number of hidden units between layers, default 8.

    Returns:

    Example"
    """
    super().__init__()

    #Use nn.linear() for creating the model parameters > this is a method to automate creation of features
    self.linear_layers = nn.Sequential(
        nn.Linear(in_features= input_features,
                  out_features= hidden_units),
        nn.ReLU(),
        nn.Linear(in_features=hidden_units,
                  out_features=hidden_units),
        nn.ReLU(),
        nn.Linear(in_features=hidden_units,
                  out_features=output_features)
    )

# Forward method to define the computation in the model
  def forward(self, x):
    return self.linear_layers(x)

# Set manual seed
torch.manual_seed(42)
torch.cuda.manual_seed(42) # if using GPU
model_0 = ClassificationModel(input_features=2,
                              output_features=4
                              hidden_units=8).to(device)
model_0

### Check model device vs target device




In [ ]:
# Check model device
next(model_0.parameters()).device

device(type='cpu')

In [ ]:
# Set model to target device
model_0.to(device)
next(model_0.parameters()).device

device(type='cpu')

### Train Model

#### Set up Loss Function & Optimiser

In [ ]:
#Loss function

loss_fn = nn.CrossEntropyLoss() #has sigmoid activation function built in

#Optimiser (stochastic gradient descent- SGD)
optimizer = torch.optim.SGD(params=model_0.parameters(),
                            lr=LEARNING_RATE) #lr = learning rate (hyperparameter) > the smaller the learning rate the smaller the change in the parameter

#### Evaluate Model accuracy

In [ ]:
def accuracy_fn(y_true, y_pred):
  correct = torch.eq(y_true, y_pred).sum().item()
  acc = (correct/len(y_pred))*100
  return acc

In [ ]:
# View outputs of forward pass
model_0.eval()
with torch.inference_mode():
  y_logits = model_0(X_test.to(device))

#Convert logits > predicted probabilities > predicted labels
  y_pred_probs = torch.softmax(y_logits, dim=1)

In [ ]:
y_pred = torch.argmax(y_pred_probs, dim=1)
y_pred

### Build a training loop and a testing loop


In [ ]:
torch.manual_seed(42)
torch.cuda.manual_seed(42) # if using GPU

# Set number of epochs

epochs = NUM_EPOCHS

# Make sure data is on correct target device
X_train = X_train.to(device)
y_train = y_train.to(device)
X_test = X_test.to(device)
y_test = y_test.to(device)

# Track different values - use these to track model performance & compare with other models
epoch_count = []
loss_values = []
test_loss_values = []

# Loop through data

### Training
# 0. Loop through the data
for epoch in range(epochs):

  #set model to training mode
  model_0.train()

  # 1. Forward pass
  y_logits = model_0(X_train).squeeze() #check squeeze
  y_pred = torch.softmax(y_logits, dim=1).argmax(dim=1)

  # 2. Calculate the loss/accuracy
  #loss = loss_fn(torch.sigmoid(y_logits),
  #              y_train) # for nn.BCELoss expects prediction probabilities as input

  loss = loss_fn(y_logits,
                 y_train)
  acc = accuracy_fn(y_true=y_train,
                    y_pred=y_pred)

  # 3. Optimiser zero grad
  optimizer.zero_grad()

  # 4. Loss backward (Backpropagation)
  loss.backward()

  # 5. Optimiser step (Gradient descent)
  optimizer.step()


### Testing

  model_0.eval()
  with torch.inference_mode(): # turns off gradient tracking

    # 1. Forward pass
    test_logits = model_0(X_test).squeeze() #check squeeze
    test_pred = torch.softmax(test_logits, dim=1).argmax(dim=1)

    # 2. Calculate the loss
    test_loss = loss_fn(test_logits
                        , y_test)
    test_acc = accuracy_fn(y_true=y_test,
                           y_pred=test_pred)

  # Print out what's happening
  if epoch % 50  == 0:

    print(f"Epoch: {epoch} | Loss:{loss:.4f} | Acc: {acc:.2f}% | Test loss:{test_loss:.4f} | Test Acc: {acc:.2f}%")


## Improve a model (from a model's perspective)

* Add more layers - gives the model more chances to learn about patterns in the data
* Add more hidden units -
* Fit for longer - increase epochs
* Change the activation function
* Change the learning rate
* Change the loss function


### Evaluate Model

In [ ]:
model_0.state_dict()

In [ ]:
# Turn model into Evaluation mode
model_0.eval()

# Make Predictions on test data
with torch.inference_mode():
  y_logits = model_0(X_test).squeeze() #check squeeze
  y_pred_probs = torch.softmax(y_logits, dim=1)

# Go from prdected probabilities to predcted labels
  y_pred = torch.argmax(y_pred_probs, dim=1)

y_pred[:10], y_test[:10]

### Classification Metrics

* Accuracy
* Precision
* F1 score
* Confusion Matrix
* Classification report

In [ ]:
!pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 961.5/961.5 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 750.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 95.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [ ]:
from torchmetrics import Accuracy

# Set up Metric
torchmetric_accuracy = Accuracy().to(device)

# Calculate accuracy
torchmetric_accuracy(y_pred, y_test)